## 從AWS Data Exchange輸出每日數據到S3存儲桶
我們從AWS Data Exchange獲取EOD股票數據並將其輸出到S3存儲桶。然後，我們對數據進行格式化，以用於我們的每日數據集。在此範例中，我們使用以下數據：https://aws.amazon.com/marketplace/pp/prodview-e2aizdzkos266

輸出數據集
包含10大美國公司之一的20年EOD數據
數據以CSV格式保存到指定的S3存儲桶中。
hist_data_daily/{sym}.csv（列：日期，股票代號，開盤價，最高價，最低價，收盤價，交易量）

In [ ]:
# get S3 bucket
s3bucket=!(aws s3 ls | grep algotrading- | awk  '{print $3}')
s3bucket=s3bucket[0]
s3bucket

In [ ]:
# symbol
sym='JNJ'

In [ ]:
# copy daily eod data to local
! aws s3 cp s3://{s3bucket}/daily_adjusted_{sym}.csv ./

In [ ]:
import pandas as pd

df = pd.read_csv("daily_adjusted_"+sym+".csv",infer_datetime_format=True, parse_dates=['timestamp'], index_col=['timestamp'])
del df["split_coefficient"]
del df["dividend_amount"]
del df["adjusted_close"]
df.rename(inplace=True,columns={'volume':'vol'})
df.index=df.index.rename('dt')
df['sym']=sym
df = df[['sym', 'open', 'high', 'low', 'close','vol']]
df.sort_index(inplace=True)
df.head()

In [ ]:
df.to_csv(sym+'.csv')

In [ ]:
!aws s3 cp {sym}.csv s3://{s3bucket}/hist_data_daily/
!rm daily_adjusted_{sym}.csv
!rm {sym}.csv